In [19]:
import pandas as pd
import json
from bs4 import BeautifulSoup as bs
import requests as rq
import bs4
import re
from selenium import webdriver
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver

In [21]:


def scraping_falabella(search_url):
    search_url=search_url
    #search_url='https://sodimac.falabella.com.pe/sodimac-pe/search?Ntt=iphone'
    products_list = []
    driver = webdriver.Chrome()
    driver.set_page_load_timeout(60)
    driver.get(search_url)
    driver.implicitly_wait(60)
    #wait = WebDriverWait(driver, 10)
    #next_button = wait.until(EC.element_to_be_clickable((By.ID, 'testId-pagination-bottom-arrow-right')))

    #Hacer clic en el botón "Next" utilizando JavaScript
    #driver.execute_script("arguments[0].click();", next_button)

    time.sleep(3)

    html=driver.page_source


    
    soup=bs(html)
    
  
    products_list_saga=[]
    productos_inicial=soup.find_all("li",{"class":"selected-mkp"})
    inicio=int(productos_inicial[-1].text)
    productos_final=soup.find_all("li",{"class":"pagination-item-mkp"})
    
    if productos_final:
        final=int(productos_final[-1].text)
        
        
    else:
        final=1
        inicio=0
    num=0
    while inicio < final:
        print(inicio,"####################################################")
        #soup=bs(html)
        
        #productos=soup.find_all('b', {"class":"pod-subTitle"})
        productos=soup.find_all('div', {"pod-layout":"2_GRID"})
        #print(productos)
        for r in productos:
            num+=1
            product_dict_saga = {
                        "Product": r.find("b",{"class":"pod-subTitle"}).text,
                        "skuId": "-",
                        "cmrPrice": "-",
                        "internetPrice": "-",
                        "normalPrice": "-",
                        "disponibilidad":num
                    }
            #print(r.find("b",{"class":"pod-subTitle"}).text)
            #VALIDACION DE TIPO DE PAGOS

            type_pago=r.find("li",{"class": "prices-0"}).attrs

            if list(type_pago)[0] =="data-cmr-price":
                crm_precio_test=r.find("li",{"class": "prices-0"}).text.split("-")[0].strip()
                #crm_precio=r.find("li",{"class": "prices-0"}).attrs.get("data-cmr-price")

                #product_dict_saga["cmrPrice"]=f"S/.{crm_precio}"
                product_dict_saga["cmrPrice"]=crm_precio_test
            if list(type_pago)[0] =="data-event-price" or list(type_pago)[0] =="data-internet-price":
                price_type = "internetPrice"
                internet_precio=r.find("li",{"class": "prices-0"}).attrs.get(list(type_pago)[0])
                product_dict_saga[price_type]=f"S/.{internet_precio}"


            type_pago_1=r.find("li",{"class": "prices-1"})
            type_pago_2=r.find("li",{"class": "prices-2"})

            if type_pago_1:
                type_pago_1=r.find("li",{"class": "prices-1"}).attrs
                if list(type_pago_1)[0] =="data-internet-price" or list(type_pago_1)[0] =="data-normal-price" or list(type_pago_1)[0] =="data-event-price":
                    if list(type_pago_1)[0] == "data-internet-price" or list(type_pago_1)[0] =="data-event-price":
                        price_type_1="internetPrice"
                    else:
                        price_type_1="normalPrice"
                    internet_normal_precio_test=" ".join(r.find("li",{"class": "prices-1"}).text.split("-")).strip()

                    #De esta manera se lista solo el primer precio
                    #internet_normal_precio=r.find("li",{"class": "prices-1"}).attrs.get(list(type_pago_1)[0])
                    #product_dict_saga[price_type_1]=f"S/.{internet_normal_precio}"

                    product_dict_saga[price_type_1]=internet_normal_precio_test


            if type_pago_2:
                type_pago_2=r.find("li",{"class": "prices-2"}).attrs
                if list(type_pago_2)[0] =="data-normal-price":
                    normal_precio=r.find("li",{"class": "prices-2"}).attrs.get(list(type_pago_2)[0])
                    product_dict_saga["normalPrice"]=f"S/.{normal_precio}"

            # RECOGER SKU
            id_sku=r.find("a",{"data-pod":"catalyst-pod"}).attrs.get("href").split("/")[-1]
            product_dict_saga["skuId"]=id_sku
            
            products_list_saga.append(product_dict_saga)
            #print(products_list_saga)

        if productos_final:
            driver.set_page_load_timeout(60)
            #driver.get(search_url)
            driver.implicitly_wait(60)
            wait = WebDriverWait(driver, 20)
            next_button = wait.until(EC.element_to_be_clickable((By.ID, 'testId-pagination-bottom-arrow-right')))

            #Hacer clic en el botón "Next" utilizando JavaScript
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(6)

            html=driver.page_source
            soup=bs(html)

        else:
            print("no hay paginacion")
                            
        
        inicio+=1

    driver.quit()


    return products_list_saga


modelo_producto = input('Ingrese el modelo del producto que desea buscar: ')

search_url='https://www.falabella.com.pe/falabella-pe/search?Ntt='+modelo_producto
data=scraping_falabella(search_url)
data



Ingrese el modelo del producto que desea buscar: iphone
1 ####################################################
2 ####################################################
3 ####################################################
4 ####################################################
5 ####################################################
6 ####################################################
7 ####################################################
8 ####################################################
9 ####################################################
10 ####################################################
11 ####################################################


[{'Product': 'Celular Apple iPhone 13 128GB',
  'skuId': '18673460',
  'cmrPrice': 'S/  2,899',
  'internetPrice': 'S/  2,999',
  'normalPrice': 'S/.4,399',
  'disponibilidad': 1},
 {'Product': 'Apple iPhone 14 128GB',
  'skuId': '19316355',
  'cmrPrice': 'S/  3,599',
  'internetPrice': 'S/  3,699',
  'normalPrice': 'S/.4,999',
  'disponibilidad': 2},
 {'Product': 'Celular Apple iPhone 13 128GB',
  'skuId': '18673460',
  'cmrPrice': 'S/  2,899',
  'internetPrice': 'S/  2,999',
  'normalPrice': 'S/.4,399',
  'disponibilidad': 3},
 {'Product': 'Apple iPhone 14 Pro Max 128GB',
  'skuId': '19316384',
  'cmrPrice': 'S/  4,699',
  'internetPrice': 'S/  4,799',
  'normalPrice': 'S/.6,899',
  'disponibilidad': 4},
 {'Product': 'Celular Apple iPhone 11 128GB',
  'skuId': '18673445',
  'cmrPrice': 'S/  1,949',
  'internetPrice': 'S/  1,999',
  'normalPrice': 'S/.3,499',
  'disponibilidad': 5},
 {'Product': 'Celular Apple iPhone 11 64GB',
  'skuId': '18673441',
  'cmrPrice': '-',
  'internetPrice

In [22]:

len(data)

561